# Test cascade failure dynamic

In [1]:
# Test cascading dynamic on a single food product

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

In [2]:
# Load data
data_trade_og = pd.read_pickle('../Data/Data_trade_filt_production.pkl')
data_prod_og = pd.read_pickle('../Data/Data_prod_filt_production.pkl')

#Extract list of all countries 
countries_iso = data_trade_og[['origin_country','origin_country_ISO']].rename(columns={'origin_country':'country','origin_country_ISO':'ISO'})
countries_iso = pd.concat([countries_iso,data_trade_og[['destin_country','destin_country_ISO']].rename(columns={'destin_country':'country','destin_country_ISO':'ISO'})])
countries_iso = countries_iso.drop_duplicates().sort_values(by='country')

len(countries_iso.country)

219

In [3]:
countries_iso

,country,ISO
96120,Afghanistan,AF
118990,Albania,AL
14665,Algeria,DZ
195495,Angola,AO
582661,Antigua and Barbuda,AG
...,...,...
21300329,Wake Island,UM-79
82137,Yemen,YE
585995,Yugoslav SFR,YU
133261,Zambia,ZM


In [4]:

# Take one product & year to work with:
data_trade = data_trade_og.loc[(data_trade_og.item_code ==15) &(data_trade_og.year ==2019)]
data_prod = data_prod_og.loc[(data_prod_og.item_code ==15) &(data_prod_og.year ==2019) ]

# 
#list_country = set(data_trade.origin_country.unique()).union(data_trade.destin_country.unique())
#list_c_codes = set(data_trade.origin_country_ISO.unique()).union(data_trade.destin_country_ISO.unique())
#c_country_map = pd.concat([list_country,list_c_codes])


In [5]:
data_prod

,area_code,area,item_code,item,element,year,unit,value
2322,2,Afghanistan,15,Wheat,Production,2019,t,4890000.00
5716,3,Albania,15,Wheat,Production,2019,t,233218.00
9231,4,Algeria,15,Wheat,Production,2019,t,3876876.00
11352,7,Angola,15,Wheat,Production,2019,t,2945.33
17139,9,Argentina,15,Wheat,Production,2019,t,19459727.00
...,...,...,...,...,...,...,...,...
527495,235,Uzbekistan,15,Wheat,Production,2019,t,6093462.00
531879,236,Venezuela (Bolivarian Republic of),15,Wheat,Production,2019,t,831.00
537477,249,Yemen,15,Wheat,Production,2019,t,100332.00
540601,251,Zambia,15,Wheat,Production,2019,t,151850.47


In [ ]:
# Find groups 
in_trade = data_trade.groupby(by = ['destin_country','year']).value.sum()
production= data_prod.groupby(by=['area','year']).value.sum()
out_product =  data_trade.groupby(by=['origin_country', 'year']).value.sum()

#data_prod.groupby(by=['area','year']).groupby(list(production.keys())[0])

# Find all pairs existing in the 3 datasets: 
all_keys = list(set(in_trade.keys()).union(production.keys()).union(out_product.keys()))
#all_keys[0]

In [ ]:
# IN-trade sum all years:  
in_trade_tot = data_trade.groupby(by = 'destin_country').value.sum()
production_tot= data_prod.groupby(by='area').value.sum()
out_product_tot =  data_trade.groupby(by='origin_country').value.sum()
all_keys_tot = list(set(in_trade_tot.keys()).union(production_tot.keys()).union(out_product_tot.keys()))
all_keys_tot

In [ ]:

def Available_product(in_trade, production, out_product, key_pair):

    imports = 0
    if key_pair in in_trade.keys() :
        imports =  in_trade[key_pair]
    
    prod = 0
    if key_pair in production.keys() :
        prod =  production[key_pair]

    available_product = imports + prod

    # output 
    exported_product = 0
    if key_pair in out_product.keys() :
        #print('output:', out_product[key_pair])
        exported_product= out_product[key_pair]

    # DOMESTIC CONSUMPTION 
    dom_con = available_product - exported_product

    if type(key_pair) is str:
        return_file =[key_pair,'All',imports, prod, available_product,exported_product,dom_con]
    else:
        return_file =[key_pair[0],key_pair[1],imports, prod, available_product,exported_product,dom_con]

    return return_file

out_data = [Available_product(in_trade, production, out_product, key_pair) for key_pair in all_keys]
data_check = pd.DataFrame(out_data,columns = ['country','year','imports','production','avail_prod','export_product','domest_consum'])

# Check total sum all years: 
out_data_tot = [Available_product(in_trade_tot, production_tot, out_product_tot, key_pair) for key_pair in all_keys_tot]
data_check_tot = pd.DataFrame(out_data_tot,columns = ['country','year','imports','production','avail_prod','export_product','domest_consum'])

In [ ]:
# Focus on the 2019 wheat trade: 
#df1= data_trade.loc[:,['origin_country_ISO','origin_country']].rename(columns={'origin_country_ISO':'ISO','origin_country': 'country'})
#df2= data_trade.loc[:,['destin_country_ISO','destin_country']].rename(columns={'destin_country_ISO':'ISO','destin_country': 'country'})
#all_iso = pd.concat([df1, df2],axis=0, ignore_index=True).drop_duplicates()

# Add domestic consumption to trade_list: 
data_agg = pd.merge(data_check, countries_iso, on = 'country')
prod_country_key = data_agg[['ISO','country']]
prod_country_key 
dom_consumption=data_agg.loc[:,['domest_consum','ISO']]
dom_consumption['destin_country_ISO'] = dom_consumption['ISO']
dom_consumption=dom_consumption.rename(columns={'ISO': 'origin_country_ISO','domest_consum':'value'})


In [ ]:
data_agg

In [ ]:
# TRADE MATRIX
df1= data_trade.loc[:,['origin_country_ISO','origin_country']].rename(columns={'origin_country_ISO':'ISO','origin_country': 'country'})
df2= data_trade.loc[:,['destin_country_ISO','destin_country']].rename(columns={'destin_country_ISO':'ISO','destin_country': 'country'})
all_iso = pd.concat([df1, df2, prod_country_key],axis=0, ignore_index=True).drop_duplicates()

df_trade = data_trade.loc[:,['origin_country_ISO','destin_country_ISO','value']]
#df_trade = pd.concat([df_trade, dom_consumption], ignore_index=True)

matrix_trade = df_trade.pivot(index = 'origin_country_ISO',columns='destin_country_ISO',values='value').fillna(0)
matrix_domestic = np.diag(dom_consumption)


# UPDATE MATRIX 
df_trade_all = pd.DataFrame(np.zeros([len(all_iso),len(all_iso)]), columns=all_iso.ISO, index=all_iso.ISO) # should be 190
df_trade_all.update(matrix_trade)
df_trade_all.rename_axis('destin_country_ISO', axis='columns',inplace=True)
df_trade_all.rename_axis('origin_country_ISO', axis='index',inplace=True)
df_trade_all
#Divide by rowsum: D= Id*(1/rowsum)
row_sum= df_trade_all.sum(axis=1) #axis =1 is rownorm

inv_sum = np.diag(1/row_sum.to_numpy())

#Multiply D*W
W = np.matmul(inv_sum,df_trade_all.to_numpy())

W_df= pd.DataFrame(W,columns=df_trade_all.columns,index=df_trade_all.index)
W_df

# Sort data agg
data_agg.index=data_agg.ISO
data_agg = data_agg.loc[all_iso.ISO,:]

In [ ]:
# update test? 
prod_vec = np.array(data_agg.avail_prod)
np.diag(prod_vec)

redivided_share = np.matmul(np.diag(prod_vec),W)

In [ ]:
# old state
og_state = df_trade_all

# 
produciton_old = data_agg.loc['RU','production']
produciton = produciton_old - 20000000

df_trade_all
production_diff = df_trade_all  


In [ ]:
long_data = df_trade_all.unstack().reset_index()
long_data.columns= ['destin_country_ISO','origin_country_ISO','value']
long_data

# in trade: 
imports =  in_trade[key_pair]


In [ ]:
inputs = long_data.groupby('destin_country_ISO').value.sum()
inputs


In [ ]:
df_trade_all

In [ ]:

def Update_states(old_trade, old_production, out_product, key_pair):
    old_trade = df_trade_all

    imports = 0
    if key_pair in in_trade.keys() :
        imports =  in_trade[key_pair]
    
    prod = 0
    if key_pair in production.keys() :
        prod =  production[key_pair]

    available_product = imports + prod

    # output 
    exported_product = 0
    if key_pair in out_product.keys() :
        #print('output:', out_product[key_pair])
        exported_product= out_product[key_pair]

    # DOMESTIC CONSUMPTION 
    dom_con = available_product - exported_product

    if type(key_pair) is str:
        return_file =[key_pair,'All',imports, prod, available_product,exported_product,dom_con]
    else:
        return_file =[key_pair[0],key_pair[1],imports, prod, available_product,exported_product,dom_con]

    return return_file


out_data = [Available_product(in_trade, production, out_product) for key_pair in all_keys]
data_check = pd.DataFrame(out_data,columns = ['country','year','imports','production','avail_prod','export_product','domest_consum'])


In [ ]:
old_trade = df_trade_all
old_trade

In [ ]:
prod_vec = np.array(data_agg.avail_prod)
np.diag(prod_vec)

redivided_share = np.matmul(np.diag(prod_vec),W)

# 

In [ ]:
product_division =W_df.loc['RU',:] * data_agg.loc['RU','avail_prod']


In [ ]:
pd.dot(prod_dc)

In [ ]:
# Sort data agg to get same order than all_iso
data_agg.index=data_agg.ISO
data_agg = data_agg.loc[all_iso.ISO,:]
prod_vec = np.array(data_agg.avail_prod)
# 
av_cons= W.mul(prod_vec)

#avail_prod = av_cons.sum(axis=1)
av_cons.sum(axis=1)

In [ ]:
data_agg

In [ ]:
W